<a href="https://colab.research.google.com/github/21-501/ds1/blob/main/i1code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Upload the CSV file
uploaded = files.upload()

# Assuming the file is named 'tasks.csv'
file_name = list(uploaded.keys())[0]

# Read the CSV file
df = pd.read_csv(file_name)

class Task:
    def __init__(self, description, priority):
        self.description = description
        self.priority = priority

    def __repr__(self):
        return f"Task(description='{self.description}', priority='{self.priority}')"

class TaskManager:
    def __init__(self, tasks):
        self.tasks = tasks

    def list_tasks(self):
        sorted_tasks = sorted(self.tasks, key=lambda x: self.priority_to_number(x.priority), reverse=True)
        return sorted_tasks

    def priority_to_number(self, priority):
        priority_map = {'High': 3, 'Medium': 2, 'Low': 1}
        return priority_map.get(priority, 0)

    def recommend_task(self, keyword):
        descriptions = [task.description for task in self.tasks]
        if not descriptions:
            return []

        # Filter tasks that contain the keyword
        keyword_tasks = [task for task in self.tasks if keyword.lower() in task.description.lower()]
        if not keyword_tasks:
            return []

        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(descriptions)
        cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

        recommendations = []
        for keyword_task in keyword_tasks:
            idx = descriptions.index(keyword_task.description)
            sim_scores = list(enumerate(cosine_similarities[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            recommended_tasks = [self.tasks[i[0]] for i in sim_scores if i[0] != idx and keyword.lower() in self.tasks[i[0]].description.lower()]
            recommendations.extend(recommended_tasks)

        # Remove duplicates while preserving order
        seen = set()
        recommendations = [task for task in recommendations if task not in seen and not seen.add(task)]
        return recommendations

# Create Task instances from the dataframe
tasks = [Task(row['description'], row['priority']) for index, row in df.iterrows()]

# Create the TaskManager instance
task_manager = TaskManager(tasks)

# List tasks
print("Listing tasks:")
for task in task_manager.list_tasks():
    print(task)

# Recommend tasks based on a keyword (example: "meeting")
keyword = "meeting"
print(f"\nRecommending tasks for keyword '{keyword}':")
recommendations = task_manager.recommend_task(keyword)
for task in recommendations:
    print(task)


Saving i1code.csv to i1code (3).csv
Listing tasks:
Task(description='Buy groceries', priority='High')
Task(description='Pay the bills', priority='High')
Task(description='Complete project report', priority='Medium')
Task(description='Prepare presentation for the meeting', priority='Medium')
Task(description='Schedule a meeting with the team', priority='Low')
Task(description='Exercise', priority='Low')

Recommending tasks for keyword 'meeting':
Task(description='Prepare presentation for the meeting', priority='Medium')
Task(description='Schedule a meeting with the team', priority='Low')
